In [5]:
import requests
import time

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [6]:
configuration = """{
    "cbc357ccb763df2852fee8c4fc7d55f2": {
        "parallelism": {
            "lowerBound": 1,
            "upperBound": 1
        },
        "resourceProfile": {
            "cpuCores": {
                "name": "CPU",
                "value": 1,
                "zero": false
            },
            "taskHeapMemory": {
                "bytes": 119453752,
                "kibiBytes": 371712,
                "mebiBytes": 363,
                "gibiBytes": 0,
                "tebiBytes": 0
            },
            "taskOffHeapMemory": {
                "bytes": 0,
                "kibiBytes": 0,
                "mebiBytes": 0,
                "gibiBytes": 0,
                "tebiBytes": 0
            },
            "managedMemory": {
                "bytes": 0,
                "kibiBytes": 351232,
                "mebiBytes": 343,
                "gibiBytes": 0,
                "tebiBytes": 0
            },
            "networkMemory": {
                "bytes": 186562644,
                "kibiBytes": 86016,
                "mebiBytes": 84,
                "gibiBytes": 0,
                "tebiBytes": 0
            }
        }
    },
    "fdaaed869c7e767a4ba54f74c039d468": {
        "parallelism": {
            "lowerBound": 1,
            "upperBound": 1
        },
        "resourceProfile": {
            "cpuCores": {
                "name": "CPU",
                "value": 1,
                "zero": false
            },
            "taskHeapMemory": {
                "bytes": 119453752,
                "kibiBytes": 371712,
                "mebiBytes": 363,
                "gibiBytes": 0,
                "tebiBytes": 0
            },
            "taskOffHeapMemory": {
                "bytes": 0,
                "kibiBytes": 0,
                "mebiBytes": 0,
                "gibiBytes": 0,
                "tebiBytes": 0
            },
            "managedMemory": {
                "bytes": 0,
                "kibiBytes": 351232,
                "mebiBytes": 343,
                "gibiBytes": 0,
                "tebiBytes": 0
            },
            "networkMemory": {
                "bytes": 186562644,
                "kibiBytes": 86016,
                "mebiBytes": 84,
                "gibiBytes": 0,
                "tebiBytes": 0
            }
        }
    },
    "afcfe443e23d73c4737dc3635c2866c0": {
        "parallelism": {
            "lowerBound": $PARALLELISM,
            "upperBound": $PARALLELISM
        },
        "resourceProfile": {
            "cpuCores": {
                "name": "CPU",
                "value": 1,
                "zero": false
            },
            "taskHeapMemory": {
                "bytes": 119453752,
                "kibiBytes": 371712,
                "mebiBytes": 363,
                "gibiBytes": 0,
                "tebiBytes": 0
            },
            "taskOffHeapMemory": {
                "bytes": 0,
                "kibiBytes": 0,
                "mebiBytes": 0,
                "gibiBytes": 0,
                "tebiBytes": 0
            },
            "managedMemory": {
                "bytes": $MANAGED,
                "kibiBytes": 351232,
                "mebiBytes": 343,
                "gibiBytes": 0,
                "tebiBytes": 0
            },
            "networkMemory": {
                "bytes": 186562644,
                "kibiBytes": 86016,
                "mebiBytes": 84,
                "gibiBytes": 0,
                "tebiBytes": 0
            }
        }
    }
}"""

In [7]:
def run_xp():
    jobs = requests.get("http://flink.127-0-0-1.sslip.io/jobs").json()

    job_id = [job['id'] for job in jobs['jobs'] if job['status'] == 'RUNNING'][0]
    
    for parallelism in [1, 2, 4, 8]:
        for managed in [128, 256, 518, 1024, 2048]:
            config = configuration.replace("$PARALLELISM", str(parallelism))
            config = config.replace("$MANAGED", str(managed * 1024 * 1024))
            #print(config)
            r = requests.put(f"http://flink.127-0-0-1.sslip.io/jobs/{job_id}/justin", data=config)
            time.sleep(600)

# Read-only

In [ ]:
!kubectl apply -f read-only/query.yaml
time.sleep(30)
run_xp()
!kubectl delete -f read-only/query.yaml


flinkdeployment.flink.apache.org/flink created


# Write-only

In [ ]:
!kubectl apply -f write-only/query.yaml
time.sleep(30)
run_xp()
!kubectl delete -f write-only/query.yaml


# Update

In [ ]:
!kubectl apply -f update/query.yaml
time.sleep(30)
run_xp()
!kubectl delete -f update/query.yaml


# Observing Throughputs

We can use Grafana to access Prometheus metrics and observe the throughput reached for each configuration.

From a terminal, we need to expose the service to the outside using the `port-forward` utility.
```bash
$ kubectl port-forward -n manager svc/prom-grafana 3000:80
```
Grafana is now available on port 3000.

[http://localhost:3000/explore?orgId=1&left=%5B%22now-3h%22,%22now%22,%22Prometheus%22,%7B%22refId%22:%22A%22,%22instant%22:true,%22range%22:true,%22exemplar%22:true,%22expr%22:%22sum(flink_taskmanager_job_task_operator_numRecordsOutPerSecond%7Boperator_name%3D~%5C%22Source.*%5C%22%7D)%22%7D%5D](http://localhost:3000/explore?orgId=1&left=%5B%22now-3h%22,%22now%22,%22Prometheus%22,%7B%22refId%22:%22A%22,%22instant%22:true,%22range%22:true,%22exemplar%22:true,%22expr%22:%22sum(flink_taskmanager_job_task_operator_numRecordsOutPerSecond%7Boperator_name%3D~%5C%22Source.*%5C%22%7D)%22%7D%5D)

Credentials are: 
- admin
- prom-operator
